Any cell that should be run everytime will have * in the top comment/line

near future upgrades : buffered write to txt file (dont want to add delay if not needed    
how do you determine what the buffer size should be and what are the units ?   
move arduino to opposite side of the rig to enable camera placement on user side  
it actuaLLY matters if mouse is holding bar when behavior starts, so we need to start behvaior before the bar is placed  
otherwise the bar touch sensor will not work until it slowly recalibrates itself   
  
also i think we could make the lines specifically print around solenoid activated bc thats what i care about seeing real time (ie is it licking?)

tomorrow morning work on the damn camera system with atlas

In [1]:
# * run at start of each session, clear all variables
for var in list(globals().keys()):
    if var not in ['__builtins__', '__name__', '__doc__', '__package__', '__loader__', '__spec__', '__annotations__']:
        del globals()[var]

In [2]:
import serial
import time 
from datetime import datetime
import os
import csv
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askdirectory

To get port number type 'ls /dev/tty.*' in terminal

Press esc M to change cell to text (markdown)
To stop cell, click outside of code section and press II quickly (not capital) 

In [31]:
# * run this cell to test behavior rig for run_time
port = '/dev/tty.usbmodem101'  # Replace with your Arduino's port
baudrate = 115200  # make sure this matches the baud rate in the Arduino sketch (.ino file)
run_time = 600 #sec
#request user for identiying information
animal_ID = input("Enter Animal ID: ")
training_stage = input("Enter training stage: ")

#open window to choose directory 
root = Tk()
root.withdraw()  # Hide the main tkinter window
save_directory = askdirectory(title="Select Directory to Save Data File")
data_filename = f"{animal_ID}_{training_stage}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
data_filepath = os.path.join(save_directory, data_filename)

#initialize file to write to as 'file'
with open(data_filepath, 'w') as file:
    #initialize serial connection
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(2) 
    
    #TRIGGER CAMERA, 2P HERE
    
    #send start command to Arduino
    start_time = time.time() #use for duration based arduino script start/stop
    ser.write(b'S') 
    behav_active = True
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    print(f'{timestamp} - Sent start command to Arduino.')
    file.write(f'{timestamp} - Sent start command to Arduino.\n')
    
    try:
        count = 0
        # Read data from Arduino
        while behav_active:
            if ser.in_waiting > 0:
                try:
                    line = ser.readline().decode('utf-8', errors='ignore').rstrip()
                    timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                    log_line = f'{timestamp} - Arduino: {line}\n'
                    # Reduce print statements to every 10th message
                    count += 1
                    if count % 10 == 0:
                        print(log_line, end='')
                    #print(log_line, end='')  # Print the data in real-time
                    file.write(log_line)  # Save data to file incrementally
                    file.flush()  # Ensure the data is written to the disk
                    os.fsync(file.fileno())  # Force writing to disk
                except UnicodeDecodeError:
                    print("Failed to decode serial data")
                    continue
    
            # Check if 20 seconds have passed to send the stop command
            if time.time() - start_time >= run_time:
                #execution line (send stop command to arduino)
                ser.write(b'X') 
                #reporting lines
                print('Sent stop command to Arduino.')
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
                file.write(f'{timestamp} - Sent stop command to Arduino.')
                #exit behavior code line
                behav_active = False
    
    except KeyboardInterrupt:
        #execution line (send stop command to arduino)
        ser.write(b'X')
        #reporting lines
        print("Interrupted by user. Sent stop command to Arduino.")
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        file.write(f'{timestamp} - Interrupted by user. Sent stop command to Arduino.')
    finally:
        #close serial connection
        ser.close()

Enter Animal ID:  806N
Enter training stage:  Lick_new3tap


2024-08-15 23:28:32.998 - Sent start command to Arduino.
23:28:33.086 - Arduino: bar: 0
23:28:33.238 - Arduino: lick: 0
23:28:33.336 - Arduino: 0
23:28:33.488 - Arduino: bar: 515
23:28:33.590 - Arduino: lick: 0
23:28:33.689 - Arduino: 0
23:28:33.820 - Arduino: bar: 501
23:28:33.914 - Arduino: lick: 0
23:28:34.004 - Arduino: 0
23:28:34.139 - Arduino: bar: 499
23:28:34.237 - Arduino: lick: 0
23:28:34.340 - Arduino: 0
23:28:34.471 - Arduino: bar: 516
23:28:34.569 - Arduino: lick: 0
23:28:34.655 - Arduino: 0
23:28:34.786 - Arduino: bar: 528
23:28:34.889 - Arduino: lick: 0
23:28:34.987 - Arduino: 0
23:28:35.106 - Arduino: bar: 101
23:28:35.204 - Arduino: lick: 0
23:28:35.306 - Arduino: 0
23:28:35.429 - Arduino: bar: 235
23:28:35.523 - Arduino: lick: 0
23:28:35.622 - Arduino: 0
23:28:35.753 - Arduino: bar: 524
23:28:35.855 - Arduino: lick: 0
23:28:35.954 - Arduino: 0
23:28:36.089 - Arduino: bar: 518
23:28:36.187 - Arduino: bar: 427
23:28:36.273 - Arduino: lick: 0
23:28:36.388 - Arduino: 0
23

In [41]:
# skip 
def get_integer_input(prompt):
    while True:
        try:
            value = int(input(prompt))
            return value
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

In [42]:
# skip only run if you want to redefine the touch duration
set_touch = get_integer_input("Enter touch duration in msec: ")

Enter touch duration in msec:  1500


In [43]:
#skip only run if you want to redefine the solenoid open duration
set_sol = get_integer_input("Enter solenoid open duration in msec: ")

Enter solenoid open duration in msec:  25


In [50]:
#skip depricated version where we try to send variable values to arduino
port = '/dev/tty.usbmodem101'  # Replace with your Arduino's port
baudrate = 115200  # make sure this matches the baud rate in the Arduino sketch (.ino file)
run_time = 30 #sec
#request user for identiying information
animal_ID = input("Enter Animal ID: ")
training_stage = input("Enter training stage: ")

#open window to choose directory 
root = Tk()
root.withdraw()  # Hide the main tkinter window
save_directory = askdirectory(title="Select Directory to Save Data File")
data_filename = f"{animal_ID}_{training_stage}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
data_filepath = os.path.join(save_directory, data_filename)

#initialize text file to write to as 'file'
with open(data_filepath, 'w') as file:
    #initialize serial connection
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(2) 
    
    #update arrduino variables if redefined by user
    if set_touch:
        ser.write(f"SET_TOUCH={set_touch}\n".encode('utf-8'))
        time.sleep(1)  
        print(f"Touch duration set to: {set_touch} msec")
        file.write(f"Touch duration set to: {set_touch} msec\n")
    if set_sol:
        ser.write(f"SET_SOL={set_sol}\n".encode('utf-8'))
        time.sleep(1)  
        print(f"Solenoid open duration set to: {set_sol} msec")
        file.write(f"Solenoid open duration set to: {set_sol} msec\n")
    
    #TRIGGER CAMERA, 2P HERE
    
    #send start command to Arduino
    start_time = time.time() #use for duration based arduino script start/stop
    ser.write("S\n".encode('utf-8')) #ser.write(b'S') char version not compatible with string arduino readout
    behav_active = True
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    print(f'{timestamp} - Sent start command to Arduino.')
    file.write(f'{timestamp} - Sent start command to Arduino.\n')
    
    try:
        # Read data from Arduino
        while behav_active:
            if ser.in_waiting > 0:
                try:
                    line = ser.readline().decode('utf-8', errors='ignore').rstrip()
                    timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                    log_line = f'{timestamp} - Arduino: {line}\n'
                    print(log_line, end='')  # Print the data in real-time
                    file.write(log_line)  # Save data to file incrementally
                    file.flush()  # Ensure the data is written to the disk
                    os.fsync(file.fileno())  # Force writing to disk
                except UnicodeDecodeError:
                    print("Failed to decode serial data")
                    continue
    
            # Check if 20 seconds have passed to send the stop command
            if time.time() - start_time >= run_time:
                #execution line (send stop command to arduino)
                ser.write("X\n".encode('utf-8')) #ser.write(b'X') char version not compatible with string arduino readout
                #reporting lines
                print('Sent stop command to Arduino.')
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
                file.write(f'{timestamp} - Sent stop command to Arduino.')
                #exit behavior code line
                behav_active = False
    
    except KeyboardInterrupt:
        #execution line (send stop command to arduino)
        ser.write("X\n".encode('utf-8')) #ser.write(b'X') char version not compatible with string arduino readout
        #reporting lines
        print("Interrupted by user. Sent stop command to Arduino.")
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        file.write(f'{timestamp} - Interrupted by user. Sent stop command to Arduino.')
    finally:
        #close serial connection
        ser.close()

Enter Animal ID:  test
Enter training stage:  lick


NameError: name 'set_touch' is not defined

In [ ]:
#depricated version where we save data to csv at end. 
#not worth bc postprocessing will be much better and this has no intermittent save data  
port = '/dev/tty.usbmodem101'  # Replace with your Arduino's port
baudrate = 115200  # make sure this matches the baud rate in the Arduino sketch (.ino file)
#request user for identiying information
#in the future commands sent to arduino can be changed based on the training stage 
animal_ID = input("Enter Animal ID: ")
training_stage = input("Enter training stage: ")

#initialize serial connection
ser = serial.Serial(port, baudrate, timeout=1)
time.sleep(2)  # wait for the connection to be established
#TRIGGER CAMERA, 2P HERE
#initialize list for sensor data storage 
sensor_data = []

#send start command to Arduino
start_time = time.time() #use for duration based arduino script start/stop
ser.write(b'S') 
behav_active = True
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
print(f'{timestamp} - Sent start command to Arduino.')

try:
    # Read data from Arduino
    while behav_active:
        if ser.in_waiting > 0:
            try:
                line = ser.readline().decode('utf-8', errors='ignore').rstrip()
                timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                print(f'{timestamp} - Arduino: {line}')
                sensor_data.append([timestamp, line])
            except UnicodeDecodeError:
                print("Failed to decode serial data")
                continue

        # Check if 20 seconds have passed to send the stop command
        if time.time() - start_time >= 20:
            ser.write(b'X')
            print('Sent stop command to Arduino.')
            behav_active = False

except KeyboardInterrupt:
    ser.write(b'X')
    print("Interrupted by user. Sent stop command to Arduino.")
finally:
    # Open a dialog to choose a directory to save the CSV file
    root = Tk()
    root.withdraw()  # Hide the main tkinter window
    save_directory = askdirectory(title="Select Directory to Save CSV File")

    if save_directory:  # Check if the user selected a directory
        csv_filename = f"{animal_ID}_{training_stage}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        csv_filepath = os.path.join(save_directory, csv_filename)

        # Save sensor data to CSV
        with open(csv_filepath, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "Sensor Data"])
            writer.writerows(sensor_data)
        print(f"Sensor data saved to {csv_filepath}")
    else:
        print("No directory selected. Data not saved.")
    #close serial connection
    ser.close()

I have decided that keeping code inside a main script is totally irrelvant in jupyter notebook
so I will instead just put the behavior triggering code within the cell directly. 

In [15]:
def main():
    animal_id = input("Enter Animal ID: ")
    session_type = input("Enter Session Type: ")

if __name__ == '__main__':
    main()

Enter Animal ID:  animal1
Enter Session Type:  lick_direct


In [16]:
animal_id = input("Enter Animal ID: ")
session_type = input("Enter Session Type: ")

Enter Animal ID:  animal1
Enter Session Type:  lick


In [ ]:
#old version, keep for history 
def main():
    port = '/dev/tty.usbmodem101'  # Replace with your Arduino's port
    baudrate = 115200
    solenoidCount = 0
    # Initialize serial connection
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(2)  # Wait for the connection to be established

    # Send start command to Arduino
    ser.write(b'S')
    print('Sent start command to Arduino.')

    try:
        # Read data from Arduino
        while True:
            if ser.in_waiting > 0:
                line = ser.readline().decode('utf-8').rstrip()
                print(f'Arduino: {line}')
                if line == 'Solenoid Activated':
                    solenoidCount+=1
                if solenoidCount >10:
                    break  # Exit loop after solenoid activation (modify as needed)

            # Check for stop condition or other conditions as needed
            # e.g., send stop command 'X' to Arduino
            # ser.write(b'X')
            # break
    except KeyboardInterrupt:
        print("Interrupted by user")
        ser.write(b'X')  # Send stop command to Arduino
    finally:
        # Close serial connection
        ser.close()

# Ensure the main function is called if this script is executed directly
if __name__ == '__main__':
    main()


In [7]:
#old calibration script, for fixed arduino script. 
#need to ensure arduino has the correct ino file uploaded. #calibrate_open_solenoid.ino
port = '/dev/tty.usbmodem101'  # Replace with your Arduino's port
baudrate = 115200
drops = 10

# Initialize serial connection
ser = serial.Serial(port, baudrate, timeout=1)
time.sleep(2)  # Wait for the connection to be established

for i in range(drops):
    ser.write(b'C')
    print(f'valve open / drop {i+1}')
    time.sleep(0.1)  # Short delay to allow the Arduino to respond
    if ser.in_waiting > 0:
        response = ser.readline().decode('utf-8').rstrip()
        print(f'Arduino: {response}')
    
    # Optional: Additional delay between drops
    time.sleep(1)
# Close serial connection
ser.close()

valve open / drop 1
Arduino: Solenoid Activated
valve open / drop 2
Arduino: Sol pin status: 1
valve open / drop 3
Arduino: Sol pin status: 1
valve open / drop 4
Arduino: Solenoid Activated
valve open / drop 5
Arduino: Sol pin status: 1
valve open / drop 6
Arduino: Sol pin status: 1
valve open / drop 7
Arduino: Solenoid Activated
valve open / drop 8
Arduino: Sol pin status: 1
valve open / drop 9
Arduino: Sol pin status: 1
valve open / drop 10
Arduino: Solenoid Activated
